In [16]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
import datetime 
import pytz
import numpy as np

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

text_data_path = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\RECON_PO.IN600.20230227230124'
# text_data_path = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\MFPORECON.IN600.20240312000434641'
# Define positions for slicing each line into tuples
positions = [(0, 6), (6, 9), (9, 15), (15, 21), (21, 27), (27, 28), (28, 30), (30, 32), (32, 33), (33, 34), (34, 37), (37, 45), (45, 46), (46, 80)]
# Open the file and read its content
with open(text_data_path, 'r', encoding='utf-8') as file:
    text_data = file.read()
def process_text_data(text_data, positions):
    # List to store extracted data as tuples
    extracted_data = []

    try:
        for line in text_data.split('\n'):
            try:
                # Slice each line based on positions and create a tuple
                line_data = tuple(line[start:end].strip() for start, end in positions)

                # Check if the tuple is not empty (contains at least one non-blank element)
                if any(field for field in line_data):
                    # Append the tuple to the list
                    extracted_data.append(line_data)

            except Exception as e:
                print(f"Error processing line: {line}. Exception: {e}")

    except Exception as e:
        print(f"Error processing text data. Error: {str(e)}")

    # Return the list of tuples
    return extracted_data


result = process_text_data(text_data, positions)
# Convert the list of tuples to a DataFrame with custom column names
df = pd.DataFrame(result, columns=[
    'PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE',
    'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG',
    'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER'
])
df = df.iloc[:5]
# Specify the table name
table_name = 'cspoms.XXPO_PO_RECON_INBOUND_STG'

request_id = 999

file_name = 'MainframeIN600'
utc_now = datetime.datetime.now(datetime.timezone.utc)
# Convert UTC time to EST
est = pytz.timezone('America/New_York')
est_time = utc_now.astimezone(est).strftime("%Y-%m-%d %H:%M:%S")
df['file_name'] = file_name
df['created_by'] = "CSPOMS"
df['creation_date'] = est_time
df['last_updated_by'] = "CSPOMS"
df['last_update_date'] = est_time
df['request_id'] = request_id
df['process_status'] = "N"
df['INBOUND_SOURCE'] = ''
df['FILLER'] = df['FILLER'].apply(lambda x: '' if '\x00' in x else x)
# Define conditions for updating 'INBOUND_SOURCE' based on the value of 'RECON_ORIGIN'
mainframe_condition = df['RECON_ORIGIN'] == 'M'
oracle_condition = df['RECON_ORIGIN'] == 'O'

# Update 'INBOUND_SOURCE' column based on conditions using numpy.where()
df['INBOUND_SOURCE'] = np.where(mainframe_condition, 'Mainframe', np.where(oracle_condition, 'Oracle', ''))

fields = [col for col in df.columns]
print(fields)
str_sub = ', '.join(['%s'] * len(fields))
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"
print("query_stg::", insert_query)
data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]
print(data_tuples)


# Execute the INSERT statement with the data
post_cur.executemany(insert_query, data_tuples)
count = post_cur.rowcount
post_conn.commit()
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')

['PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE', 'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG', 'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER', 'file_name', 'created_by', 'creation_date', 'last_updated_by', 'last_update_date', 'request_id', 'process_status', 'INBOUND_SOURCE']
query_stg:: INSERT INTO cspoms.XXPO_PO_RECON_INBOUND_STG (PO_NUM, FACILITY, ORDER_DATE, VENDOR_NUM, DUE_DATE, PERISH_PO, PO_TYPE, BIC_BUYER_ID, PO_RECEIVED, PULLBACK_FLAG, PO_OSS_DEST, SECONDARY_PO, RECON_ORIGIN, FILLER, file_name, created_by, creation_date, last_updated_by, last_update_date, request_id, process_status, INBOUND_SOURCE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
[('815966', '904', '220801', '137693', '230228', 'N', '', 'JZ', '', 'N', '', '94187636', 'O', '', 'MainframeIN600', 'CSPOMS', '2024-03-18 06:21:37', 'CSPOMS', '2024-03-18 06:21:37', 999, 'N', 'Oracle'), ('245345', '252', '221010', '4282', '2

In [18]:
#total number of records inserted in STG table
ins_rcd = f"""select count(*) from {table_name} where request_id = {request_id}"""
post_cur.execute(ins_rcd)
inserted_records_count = post_cur.fetchone()[0]
print(f"Total number of records interted into STG {table_name} from files ::  {inserted_records_count}")
cspoms_table = 'XXPO_HEADERS_ALL' 
bb_deal_check = f"""
                    UPDATE XXPO_PO_RECON_INBOUND_STG
                    SET 
                        process_status = 'C',
                        error_message = 
                            CASE 
                                WHEN recon_origin = 'O' THEN 'PO in Oracle but NOT in CSPOMS'
                                WHEN recon_origin = 'M' THEN 'PO in MF but NOT in CSPOMS'
                                ELSE 'not in CSPOMS' -- Default error message if recon_origin is neither 'O' nor 'M'
                            END
                    WHERE 
                        po_num::varchar NOT IN (
                            SELECT ha.po_num
                            FROM cspoms.XXPO_HEADERS_ALL ha
                            WHERE po_status = 'OPEN' AND received_date IS NULL

                        )
                """
try:
    post_cur.execute(bb_deal_check)
    bb_records_count = post_cur.rowcount
    print(f"Records are not there in CSPOMS : {bb_records_count}")
    post_conn.commit()
except Exception as e:
    print("An error occurred while executing the query:", str(e))


Total number of records interted into STG cspoms.XXPO_PO_RECON_INBOUND_STG from files ::  15
Records are not there for BB Deal and OI Deal in CSPOMS : 7


In [5]:
import datetime 
import pytz
import pandas as pd

text_data_path = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\RECON_PO.IN600.20230227230124'

# Define positions for slicing each line into tuples
positions = [(0, 6), (6, 9), (9, 15), (15, 21), (21, 27), (27, 28), (28, 30), (30, 32), (32, 33), (33, 34), (34, 37), (37, 45), (45, 46), (46, 80)]
# Open the file and read its content
with open(text_data_path, 'r', encoding='utf-8') as file:
    text_data = file.read()
def process_text_data(text_data, positions):
    # List to store extracted data as tuples
    extracted_data = []

    try:
        for line in text_data.split('\n'):
            try:
                # Slice each line based on positions and create a tuple
                line_data = tuple(line[start:end].strip() for start, end in positions)

                # Check if the tuple is not empty (contains at least one non-blank element)
                if any(field for field in line_data):
                    # Append the tuple to the list
                    extracted_data.append(line_data)

            except Exception as e:
                print(f"Error processing line: {line}. Exception: {e}")

    except Exception as e:
        print(f"Error processing text data. Error: {str(e)}")

    # Return the list of tuples
    return extracted_data


result = process_text_data(text_data, positions)
# print(result)
# Convert the list of tuples to a DataFrame with custom column names
df = pd.DataFrame(result,
                  columns=
                  [
                    'PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE',
                    'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG',
                    'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER'
                  ])

# Display the DataFrame
# print(df)
df = df.iloc[:5]
df
# Specify the table name
table_name = 'cspoms.XXPO_PO_RECON_INBOUND_STG'

request_id = 111

file_name = 'OracleIN600'
utc_now = datetime.datetime.now(datetime.timezone.utc)
# Convert UTC time to EST
est = pytz.timezone('America/New_York')
est_time = utc_now.astimezone(est).strftime("%Y-%m-%d %H:%M:%S")
df['file_name'] = file_name
df['created_by'] = "CSPOMS"
df['creation_date'] = est_time
df['last_updated_by'] = "CSPOMS"
df['last_update_date'] = est_time
df['request_id'] = request_id
df['process_status'] = "N"

fields = [col for col in df.columns]
print(fields)
str_sub = ', '.join(['%s'] * len(fields))
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"
print("query_stg::", insert_query)
# Convert the DataFrame to a list of tuples for bulk insertion
df.replace('', None, inplace=True)

# df['PO_OSS_DEST'].replace('',None,inplace=True)
# df['PO_OSS_DEST'] = pd.to_numeric(df['PO_OSS_DEST'], errors='coerce')

# PO_NUM, FACILITY, ORDER_DATE, VENDOR_NUM, DUE_DATE, PERISH_PO, PO_TYPE, BIC_BUYER_ID, PO_RECEIVED, PULLBACK_FLAG, PO_OSS_DEST, SECONDARY_PO, RECON_ORIGIN, FILLER, file_name, created_by, creation_date, last_updated_by, last_update_date, request_id, process_status) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]
print(data_tuples)


# # Execute the INSERT statement with the data
# post_cur.executemany(insert_query, data_tuples)
# count = post_cur.rowcount

# print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')


['PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE', 'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG', 'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER', 'file_name', 'created_by', 'creation_date', 'last_updated_by', 'last_update_date', 'request_id', 'process_status']
query_stg:: INSERT INTO cspoms.XXPO_PO_RECON_INBOUND_STG (PO_NUM, FACILITY, ORDER_DATE, VENDOR_NUM, DUE_DATE, PERISH_PO, PO_TYPE, BIC_BUYER_ID, PO_RECEIVED, PULLBACK_FLAG, PO_OSS_DEST, SECONDARY_PO, RECON_ORIGIN, FILLER, file_name, created_by, creation_date, last_updated_by, last_update_date, request_id, process_status) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
[('815966', '904', '220801', '137693', '230228', 'N', '', 'JZ', '', 'N', '', '94187636', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N'), ('245345', '252', '221010', '4282', '230316', 'N', '', 'IH', '', 'N', '', '20076575', 'O'

In [6]:
fields = [col for col in df.columns]
print(fields)

['PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE', 'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG', 'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER', 'file_name', 'created_by', 'creation_date', 'last_updated_by', 'last_update_date', 'request_id', 'process_status']


In [ ]:
try:
    # Intentionally provide an incorrect file path
    incorrect_path = 'D:\\W\\RELEX\\RELEX\\I1674 - Po recon\\nonexistent_file.txt'
    with open(incorrect_path, 'r', encoding='utf-8') as file:
        text_data = file.read()

    result = process_text_data(text_data, positions)
    # Continue with the rest of your code
    df = pd.DataFrame(result, columns=[
        'PO_NUM', 'FACILITY', 'ORDER_DATE', 'VENDOR_NUM', 'DUE_DATE',
        'PERISH_PO', 'PO_TYPE', 'BIC_BUYER_ID', 'PO_RECEIVED', 'PULLBACK_FLAG',
        'PO_OSS_DEST', 'SECONDARY_PO', 'RECON_ORIGIN', 'FILLER'
    ])
    # Rest of the code...
except FileNotFoundError as e:
    print(f"File not found. Exception: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [4]:

data_tuples
df = df[['PO_NUM','FACILITY']].iloc[5]
df = pd.DataFrame(df)
df

NameError: name 'data_tuples' is not defined

In [13]:
try:
    # Execute the INSERT statement with the data
    post_cur.executemany(insert_query, data_tuples)
    count = post_cur.rowcount
    print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')

except Exception as e:
    # Print a general error message
    print(f'Error inserting data into the database. Exception: {e}')

    # Iterate over the data tuples to find the problematic row
    for index, row in enumerate(data_tuples):
        try:
            # Attempt to execute the INSERT statement for the current row
            post_cur.execute(insert_query, row)
        except Exception as inner_e:
            # Print the specific error for the current row
            print(f'Error inserting data for row {index + 1}. Exception: {inner_e}')


KeyboardInterrupt: 

In [25]:
# Replace empty strings with None
df.replace('', None, inplace=True)
print(df)
# Convert the DataFrame to a list of tuples for bulk insertion
data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]
# Execute the INSERT statement with the data
post_cur.executemany(insert_query, data_tuples)
post_conn.commit()
count = post_cur.rowcount
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')

   PO_NUM FACILITY ORDER_DATE VENDOR_NUM DUE_DATE PERISH_PO PO_TYPE  \
0  815966      904     220801     137693   230228         N           
1  245345      252     221010       4282   230316         N           
2  251735      905     221012     101046   230308         N           
3  255347      252     221014       3780   230303         N           
4  258466      252     221017       3067   230302         N           

  BIC_BUYER_ID PO_RECEIVED PULLBACK_FLAG  ... SECONDARY_PO RECON_ORIGIN  \
0           JZ                         N  ...     94187636            O   
1           IH                         N  ...     20076575            O   
2           KB                         N  ...     94192285            O   
3           IH                         N  ...     20076823            O   
4           IH                         N  ...     20076907            O   

  FILLER    file_name created_by        creation_date last_updated_by  \
0         OracleIN600     CSPOMS  2024-03-11 05:5

In [1]:
pip install reportlab


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

# Function to create a PDF report for error records
def create_pdf_error_report(data, filename):
    doc = SimpleDocTemplate(filename, pagesize=letter)
    elements = []

    # Define table data
    table_data = [
        ["ORDER", "SECONDARY", "ORIGIN", "DST", "TP", "ORDERED", "ARRIVAL", "APPT", "BYR", "MESSAGE"]
    ]
    for row in data:
        if "ERROR" in row[9]:
            table_data.append(row[:9] + [row[9]])

    # Create table
    table = Table(table_data)

    # Add style to table
    style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.gray),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black)])

    table.setStyle(style)
    elements.append(table)

    # Build PDF
    doc.build(elements)

# Generate the error report
data_tuples = [
    ('815966', '904', '220801', '137693', '230228', 'N', '', 'JZ', '', 'N', '', '94187636', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N'), 
    ('245345', '252', '221010', '4282', '230316', 'N', '', 'IH', '', 'N', '', '20076575', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N'), 
    ('251735', '905', '221012', '101046', '230308', 'N', '', 'KB', '', 'N', '', '94192285', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N'), 
    ('255347', '252', '221014', '3780', '230303', 'N', '', 'IH', '', 'N', '', '20076823', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N'), 
    ('258466', '252', '221017', '3067', '230302', 'N', '', 'IH', '', 'N', '', '20076907', 'O', '', 'OracleIN600', 'CSPOMS', '2024-04-02 08:16:41', 'CSPOMS', '2024-04-02 08:16:41', 111, 'N')
]

# Create PDF report
create_pdf_error_report(data_tuples, "error_report.pdf")
print("PDF report generated successfully.")


PDF report generated successfully.
